In [1]:
!pip install transformers datasets seqeval
!pip install torch --upgrade


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from transformers import XLMRobertaTokenizerFast
from datasets import Dataset, Features, Sequence, Value
from transformers import TrainingArguments
from transformers import XLMRobertaForTokenClassification, AutoModelForTokenClassification, AutoTokenizer, Trainer

In [4]:
# Function to load CoNLL formatted data
def load_conll(file_path):
    sentences = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as f:
        sentence = []
        label = []
        for line in f:
            if line.strip():  # Non-empty line
                token, label_item = line.split()
                sentence.append(token)
                label.append(label_item)
            else:  # Empty line indicates end of a sentence
                sentences.append(sentence)
                labels.append(label)
                sentence = []
                label = []
    return pd.DataFrame({'tokens': sentences, 'labels': labels})
# Load the CoNLL file
df = load_conll('/content/labeled_data.conll')


In [5]:
# Explore the first few rows
df.head()

,tokens,labels
0,"[puma, ca, pro, classic, 3400, birr, ሜክሲኮ, ኮሜር...","[B-Product, I-Product, I-Product, I-Product, B..."
1,"[goupu, lace, comfort, 3500, birr, ሜክሲኮ, ኮሜርስ,...","[B-Product, I-Product, I-Product, B-PRICE, I-P..."
2,"[chelsea, 05, half, boots, leather, 4000, birr...","[B-Product, I-Product, I-Product, I-Product, I..."
3,"[nike, fc, classic, 3300, birr, ሜክሲኮ, ኮሜርስ, ጀር...","[B-Product, I-Product, I-Product, B-PRICE, I-P..."
4,"[goupu, leather, comfort, 3500, birr, ሜክሲኮ, ኮሜ...","[B-Product, I-Product, I-Product, B-PRICE, I-P..."


In [6]:
unique_labels = set(label for sublist in df['labels'] for label in sublist)
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

In [7]:
unique_labels

{'B-LOC', 'B-PRICE', 'B-Product', 'I-LOC', 'I-PRICE', 'I-Product'}

In [8]:
df['labels'] = df['labels'].apply(lambda x: [label2id[label] for label in x])

In [9]:
# Convert DataFrame to Hugging Face Dataset
# Make sure 'labels' is a list of lists
# Define the features with the correct data types
features = Features({
    'tokens': Sequence(Value('string')),  # List of strings for tokens
    'labels': Sequence(Value('int32'))    # List of integers for labels
})

# Convert DataFrame to Hugging Face Dataset with specified features
dataset = Dataset.from_pandas(df[['tokens', 'labels']], features=features)

In [10]:
# Initialize the Fast Tokenizer
# Use the fast tokenizer
# For XLM-Roberta
tokenizer = XLMRobertaTokenizerFast.from_pretrained(
    "xlm-roberta-base",
    clean_up_tokenization_spaces=True
    )

# Define tokenizer function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",  # Padding to max length
        max_length=128  # Adjust as needed
    )

    labels = []
    for i, label in enumerate(examples['labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Get word ids for each token
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                # Token corresponds to special tokens like [CLS], [SEP], etc.
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # The first token of a word
                label_ids.append(label[word_idx])
            else:
                # Subword token, assign -100 so it's ignored during training
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

# Tokenize the dataset using xlrm_berta

tokenized_xlm_dataset = dataset.map(tokenize_and_align_labels, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

Map:   0%|          | 0/2679 [00:00<?, ? examples/s]

In [11]:
# For mBERT
tokenizer_mbert = AutoTokenizer.from_pretrained(
    'bert-base-multilingual-cased',
    clean_up_tokenization_spaces=True
    )
# Define tokenizer function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer_mbert(
        examples['tokens'],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",  # Padding to max length
        max_length=128  # Adjust as needed
    )

    labels = []
    for i, label in enumerate(examples['labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Get word ids for each token
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                # Token corresponds to special tokens like [CLS], [SEP], etc.
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # The first token of a word
                label_ids.append(label[word_idx])
            else:
                # Subword token, assign -100 so it's ignored during training
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

# Tokenize the dataset using xlrm_berta

tokenized_mbert_dataset = dataset.map(tokenize_and_align_labels, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Map:   0%|          | 0/2679 [00:00<?, ? examples/s]

In [12]:
#Tokenize the dataset using DistilBERT
# For DistilBERT
tokenizer_distilbert = AutoTokenizer.from_pretrained(
    'distilbert-base-multilingual-cased',
    clean_up_tokenization_spaces=True
    )
# Define tokenizer function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer_distilbert(
        examples['tokens'],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",  # Padding to max length
        max_length=128  # Adjust as needed
    )

    labels = []
    for i, label in enumerate(examples['labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Get word ids for each token
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                # Token corresponds to special tokens like [CLS], [SEP], etc.
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                # The first token of a word
                label_ids.append(label[word_idx])
            else:
                # Subword token, assign -100 so it's ignored during training
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

tokenized_distilbert_dataset = dataset.map(tokenize_and_align_labels, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Map:   0%|          | 0/2679 [00:00<?, ? examples/s]

In [13]:
# Split into train and validation datasets
train_test_split_xlm = tokenized_xlm_dataset.train_test_split(test_size=0.1)  # 90% train, 10% validation
train_test_split_mbert = tokenized_mbert_dataset.train_test_split(test_size=0.1)  # 90% train, 10% validation
train_test_split_distilbert = tokenized_distilbert_dataset.train_test_split(test_size=0.1)  # 90% train, 10% validation

In [16]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",     # Evaluates at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,   # Reduced batch size
    per_device_eval_batch_size=8,    # Reduced evaluation batch size
    num_train_epochs=2,              # Fewer epochs
    weight_decay=0.01,               # Strength of weight decay
    max_grad_norm=None,              # Disable gradient clipping
    logging_dir='./logs',            # Directory for storing logs
    logging_strategy="steps",        # Log at regular intervals
    logging_steps=100,               # Log every 100 steps
    save_strategy="epoch",           # Save model at the end of each epoch
    report_to="none",                # Only show logs in the output
    use_cpu=False,                   # Switch to GPU if available
    load_best_model_at_end=True,     # Load the best model at the end
    metric_for_best_model="eval_loss",
    save_total_limit=1               # Only keep the best model
)


In [17]:
# Initialize each of the models
# For XLM-Roberta
model_xlmr = XLMRobertaForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=len(unique_labels)) # Ensure unique_labels is defined

# For DistilBERT
model_distilbert = AutoModelForTokenClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=len(unique_labels))

# For mBERT
model_distilbert = AutoModelForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(unique_labels))

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Define the function to compute evaluation metrics
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_labels = [[label for label in label_row if label != -100] for label_row in labels]
    predicted_labels = [[pred for pred, true in zip(pred_row, true_row) if true != -100] for pred_row, true_row in zip(predictions, labels)]

    # Flatten the lists
    true_labels_flat = [item for sublist in true_labels for item in sublist]
    predicted_labels_flat = [item for sublist in predicted_labels for item in sublist]

    # Calculate metrics
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels_flat, predicted_labels_flat, average='weighted', zero_division=True)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [19]:
trainer_xlmr = Trainer(
    model=model_xlmr,
    args=training_args,
    train_dataset=train_test_split_xlm['train'],
    eval_dataset=train_test_split_xlm['test'],  # Changed from validation to test based on split
    compute_metrics=compute_metrics, # compute f1-score, precision and recall
)
trainer_distilbert = Trainer(
    model=model_distilbert,
    args=training_args,
    train_dataset=train_test_split_mbert['train'],
    eval_dataset=train_test_split_mbert['test'],  # Changed from validation to test based on split
    compute_metrics=compute_metrics, # compute f1-score, precision and recall
)
trainer_mbert = Trainer(
    model=model_distilbert,
    args=training_args,
    train_dataset=train_test_split_distilbert['train'],
    eval_dataset=train_test_split_distilbert['test'],  # Changed from validation to test based on split
    compute_metrics=compute_metrics, # compute f1-score, precision and recall
)

In [20]:
# Fine-tune mBERT
print('For mBERT')
trainer_mbert.train()

For mBERT


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.006800,0.003075,0.999744,0.999743,0.999743
2,0.001300,0.002766,0.999744,0.999743,0.999743


TrainOutput(global_step=604, training_loss=0.028653658638917997, metrics={'train_runtime': 243.9436, 'train_samples_per_second': 19.767, 'train_steps_per_second': 2.476, 'total_flos': 315004631583744.0, 'train_loss': 0.028653658638917997, 'epoch': 2.0})

In [21]:
# Fine-tune XLM-Roberta
print('For XLM-Roberta')
trainer_xlmr.train()




For XLM-Roberta


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.004700,0.006802,0.999222,0.999222,0.999222
2,0.002000,0.006653,0.999222,0.999222,0.999222


TrainOutput(global_step=604, training_loss=0.04791378312195275, metrics={'train_runtime': 587.9326, 'train_samples_per_second': 8.202, 'train_steps_per_second': 1.027, 'total_flos': 315004631583744.0, 'train_loss': 0.04791378312195275, 'epoch': 2.0})

In [22]:
# Fine-tune XLM-Roberta
print('For XLM-Roberta')
trainer_xlmr.train()

For XLM-Roberta


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.001000,0.006884,0.999222,0.999222,0.999222
2,0.001200,0.006907,0.999222,0.999222,0.999222


TrainOutput(global_step=604, training_loss=0.0014969022757225726, metrics={'train_runtime': 494.0339, 'train_samples_per_second': 9.76, 'train_steps_per_second': 1.223, 'total_flos': 315004631583744.0, 'train_loss': 0.0014969022757225726, 'epoch': 2.0})

In [1]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Get true and predicted labels for the test dataset
true_labels = [label for row in test_dataset for label in row['labels']]
predicted_labels = [np.argmax(logits, axis=-1) for logits in predictions]

# Create the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)

plt.title('Confusion Matrix for Entity Recognition')
plt.show()


NameError: name 'test_dataset' is not defined

In [32]:
# Create a dictionary to hold the results
results = {}

# Evaluate each model and store the results
results['XLM-Roberta'] = trainer_xlmr.evaluate()
results['DistilBERT'] = trainer_distilbert.evaluate()
results['mBERT'] = trainer_mbert.evaluate()

# Extracting the relevant metrics
eval_loss = [results[model]['eval_loss'] for model in results]
eval_runtime = [results[model]['eval_runtime'] for model in results]
samples_per_second = [results[model]['eval_samples_per_second'] for model in results]

# Visualization for Evaluation Loss
plt.figure(figsize=(12, 4))

# Plotting Evaluation Loss
plt.subplot(1, 3, 1)
plt.bar(results.keys(), eval_loss, color=['blue', 'orange', 'green'])
plt.xlabel('Models')
plt.ylabel('Evaluation Loss')
plt.title('Evaluation Loss for Different Models')

# Plotting Evaluation Runtime
plt.subplot(1, 3, 2)
plt.bar(results.keys(), eval_runtime, color=['blue', 'orange', 'green'])
plt.xlabel('Models')
plt.ylabel('Evaluation Runtime (seconds)')
plt.title('Evaluation Runtime for Different Models')

# Plotting Samples Per Second
# plt.subplot(1, 3, 3)
# plt.bar(results.keys(), samples_per_second, color=['blue', 'orange', 'green'])
# plt.xlabel('Models')
# plt.ylabel('Samples Per Second')
# plt.title('Samples Per Second for Different Models')

plt.tight_layout()
plt.show()

AttributeError: `AcceleratorState` object has no attribute `distributed_type`. This happens if `AcceleratorState._reset_state()` was called and an `Accelerator` or `PartialState` was not reinitialized.

In [30]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=50,
    report_to="none",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_cpu=True  # Force CPU if necessary
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
trainer_xlmr = Trainer(
    model=xlmr_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=xlmr_tokenizer
)

trainer_distilbert = Trainer(
    model=distilbert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=distilbert_tokenizer
)

trainer_mbert = Trainer(
    model=mbert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=mbert_tokenizer
)


NameError: name 'xlmr_model' is not defined